In [2]:
import plotly
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.plotly as py

from ipywidgets import widgets

In [3]:
asndf = pd.read_csv(
    'abuse-with-asn-and-geo.csv')
ccdf = pd.read_csv('asn-cc-mapping.csv', sep="|")

ccdf = ccdf.drop(columns=['CC'])

df = asndf.merge(ccdf, on='ASN')

df['ASN'] = df['ASN'].astype(str)
df = df.sort_values(by=['abusePercent'])

In [40]:
countries = list(df['CC'].unique())
countries.sort()
countries.insert(0, "All")

origin = widgets.Dropdown(
    options=countries,
    value='All',
    description='Country Code:',
)


firstdf = df.nlargest(50, 'abusePercent')
# Assign an emptry figure widget with two traces
trace1 = go.Bar(x=firstdf['ASN'].astype(str), y=firstdf['abusePercent'])

layout = go.Layout(
                        title=dict(
                            text='Top 50 ASNs by Abuse Percentage'
                        ),
                        xaxis=dict(
                            type="category",
                            title='AS Number'
                        ),
                        yaxis=dict(
                            title='% of Abusive IPs',
                            categoryorder = "category ascending"
                        ),
                        barmode='overlay'
                    )

tabletrace = go.Table(
    header=dict(values=list(['ASN', 'AS Name']),
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[firstdf.ASN, firstdf.AS_Name],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

layout2 = go.Layout(
                        title=dict(
                            text='ASN to AS Name Mapping'
                        ))

g = go.FigureWidget(data=[trace1], layout=layout)
h = go.FigureWidget(data=[tabletrace], layout=layout2)

In [41]:
def validate():
    if origin.value in df['CC'].unique():
        return True
    if origin.value == "All":
        return True
    else:
        return False


def response(change):
    if validate():
        if origin.value == "All":
            temp_df = df
        else:
            filter_list = df['CC'] == origin.value
            temp_df = df[filter_list]
        top50df = temp_df.nlargest(50, 'abusePercent')
        x1 = top50df['ASN'].astype(str)
        y1 = top50df['abusePercent']
        z1 = top50df['AS_Name']
 
#        bar = g.data[0]
#        bar.x = x1
#        bar.y = y1
        
        table = h.data[0]
        table = [x1, z1]
        with g.batch_update():
            g.data[0].x = x1
            g.data[0].y = y1

        with h.batch_update():
            h.data[0].cells.values = [top50df.ASN, top50df.AS_Name]
            
        g.layout = layout

origin.observe(response, names="value")


In [42]:
container2 = widgets.HBox([origin])
widgets.VBox([container2,
              g, h])

In [37]:
h.data.cells.values



AttributeError: 'tuple' object has no attribute 'cells'

In [27]:
top50df

NameError: name 'top50df' is not defined